Setup

Installs and imports

Install and import TensorFlow and dependencies:

In [1]:
!pip install -q h5py pyyaml 

In [2]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras

tf.__version___

'1.11.0'

In [3]:
(train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.mnist.load_data()
train_labels = train_labels[:10000]
test_labels = test_labels[:10000]

train_data[0].shape

(28, 28)

In [6]:
train_data = train_data[:10000].reshape(-1, 28 * 28) / 255.0
test_data = test_data[:10000].reshape(-1, 28 * 28) / 255.0

In [7]:
train_data[0].shape

(784,)

In [9]:
#returns a short sequential model
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(28*28, )),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation=tf.nn.softmax)
        
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=["accuracy"])
    return model

#Create model
model = create_model()
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Save checkpoints during training

The primary use case is to automatically save checkpoints during and at the end of training. This way you can use a trained model without having to retrain it, or pick-up training where you left of—in case the training process was interrupted.

In [11]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)


model.fit(train_data, train_labels,  epochs = 10, 
          validation_data = (test_data,test_labels),
          callbacks = [cp_callback])  # pass callback to training

Train on 10000 samples, validate on 10000 samples
Epoch 1/10
 9792/10000 [============================>.] - ETA: 0s - loss: 1.9360 - acc: 0.4994
Epoch 00001: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
10000/10000 [==============================] - 2s 180us/step - loss: 1.9259 - acc: 0.5017 - val_loss: 1.3768 - val_acc: 0.6333
Epoch 2/10
 9984/10000 [============================>.] - ETA: 0s - loss: 1.0592 - acc: 0.7426
Epoch 00002: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
10000/10000 [==============================] - 1s 146us/step - loss: 1.0592 - acc: 0.7425 - val_loss: 0.8360 - val_acc: 0.8122
Epoch 3/10
 9952/10000 [============================>.] - ETA: 0s - loss: 0.7260 - acc: 0.8260
Epoch 00003: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
10000/10000 [==============================] - 1s 145us/step - loss: 0.7256 - acc: 0.8260 - val_los

In [12]:
!ls {checkpoint_dir}

checkpoint                  cp.ckpt.index
cp.ckpt.data-00000-of-00001


In [13]:
loss, acc = model.evaluate(test_data, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

10000/10000 [==============================] - 0s 27us/step
Untrained model, accuracy: 90.09%


Then load the weights from the checkpoint, and re-evaluate:

In [15]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_data, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

10000/10000 [==============================] - 0s 34us/step
Restored model, accuracy: 90.09%


Checkpoint callback options

The callback provides several options to give the resulting checkpoints unique names, and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every 5-epochs:

In [18]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    #Save weights, every 5-epochs.
    period=5
)

model = create_model()
model.fit(train_data, train_labels,
         epochs = 50, callbacks = [cp_callback],
         validation_data = (test_data, test_labels), verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00010: saving model to training_2/cp-0010.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00015: saving model to training_2/cp-0015.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00020: saving model to training_2/cp-0020.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00025: saving model to training_2/cp-0025.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00030: saving model to training_2/cp-0030.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00035: saving model to training_2/cp-0035.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00040: saving model to training_2/cp-0040.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00045: saving model to training_2/cp-0045.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 000

Now, have a look at the resulting checkpoints (sorting by modification date):

In [19]:
import pathlib

# Sort the checkpoints by modification time.
checkpoints = pathlib.Path(checkpoint_dir).glob("*.index")
checkpoints = sorted(checkpoints, key=lambda cp:cp.stat().st_mtime)
checkpoints = [cp.with_suffix('') for cp in checkpoints]
latest = str(checkpoints[-1])
checkpoints

[PosixPath('training_2/cp-0005.ckpt'),
 PosixPath('training_2/cp-0010.ckpt'),
 PosixPath('training_2/cp-0015.ckpt'),
 PosixPath('training_2/cp-0020.ckpt'),
 PosixPath('training_2/cp-0025.ckpt'),
 PosixPath('training_2/cp-0030.ckpt'),
 PosixPath('training_2/cp-0035.ckpt'),
 PosixPath('training_2/cp-0040.ckpt'),
 PosixPath('training_2/cp-0045.ckpt'),
 PosixPath('training_2/cp-0050.ckpt')]

To test, reset the model and load the latest checkpoint:

In [22]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_data, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

10000/10000 [==============================] - 0s 35us/step
Restored model, accuracy: 93.76%


Manually save weights

Above you saw how to load the weights into a model.

Manually saving the weights is just as simple, use the Model.save_weights method.

In [23]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_data, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


Consider using a TensorFlow optimizer from `tf.train`.
10000/10000 [==============================] - 0s 37us/step
Restored model, accuracy: 93.76%


Save the entire model

The entire model can be saved to a file that contains the weight values, the model's configuration, and even the optimizer's configuration. This allows you to checkpoint a model and resume training later—from the exact same state—without access to the original code.

Saving a fully-functional model in Keras is very useful—you can load them in TensorFlow.js and then train and run them in web browsers.

Keras provides a basic save format using the HDF5 standard. For our purposes, the saved model can be treated as a single binary blob.

In [24]:
model = create_model()

model.fit(train_data, train_labels, epochs=5)

# Save entire model to a HDF5 file
model.save('my_model.h5')


Epoch 1/5
10000/10000 [==============================] - 2s 161us/step - loss: 1.9284 - acc: 0.5299
Epoch 2/5
10000/10000 [==============================] - 1s 124us/step - loss: 1.0581 - acc: 0.7430
Epoch 3/5
10000/10000 [==============================] - 1s 127us/step - loss: 0.7265 - acc: 0.8239
Epoch 4/5
10000/10000 [==============================] - 1s 128us/step - loss: 0.5787 - acc: 0.8555
Epoch 5/5
10000/10000 [==============================] - 1s 129us/step - loss: 0.4994 - acc: 0.8715


In [26]:
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [27]:
loss, acc = new_model.evaluate(test_data, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

10000/10000 [==============================] - 0s 41us/step
Restored model, accuracy: 87.97%


This technique saves everything:

* The weight values

* The model's configuration(architecture)

* The optimizer configuration

Keras saves models by inspecting the architecture. Currently, it is not able to save TensorFlow optimizers (from tf.train). When using those you will need to re-compile the model after loading, and you will loose the state of the optimizer.